In [1]:
import numpy as np
import pandas as pd
import json

pd.set_option('display.max_columns', 500)

In [9]:
df = pd.read_csv('train_clean.csv')

/home/bread/anaconda3/envs/pytorch/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cont = ['visitNumber', 'visitStartTime', 'hits', 'pageviews', 'transactionRevenue']
cat = ['newVisits', 'bounces', 'channelGrouping', 'browser', 'operatingSystem', 'isMobile', 
       'deviceCategory', 'continent', 'subContinent', 'country', 'region', 'city', 'networkDomain']

In [4]:
train_df = pd.DataFrame(np.zeros((len(df), len(cont)+len(cat)+1)), columns=(cont+cat+['fullVisitorId']))

In [5]:
df['transactionRevenue'] = np.log(df['transactionRevenue'].values)
df = df.replace(np.log(0), 0)
tr = {'mean': 0, 'std': 0}
for col in cont:
    print(col)
    m = df[col].values.mean()
    sd = df[col].values.std()
    if(col == 'transactionRevenue'):
        tr['mean'] = m
        tr['std'] = sd
    train_df[col] = (df[col].values-m) / sd
for col in cat:
    train_df[col] = df[col].values
train_df['fullVisitorId'] = df['fullVisitorId'].values

/home/bread/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/home/bread/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


visitNumber
visitStartTime
hits
pageviews
transactionRevenue


In [6]:
train_df.head()

,visitNumber,visitStartTime,hits,pageviews,transactionRevenue,newVisits,bounces,channelGrouping,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,city,networkDomain,fullVisitorId
0,-0.136249,-1.349672,-0.373029,-0.4056,-0.113349,1,1,Organic Search,Chrome,Windows,False,desktop,Asia,Western Asia,Turkey,Izmir,Izmir,ttnet.com.tr,1131660440785968503
1,-0.136249,-1.344157,-0.373029,-0.4056,-0.113349,1,1,Organic Search,Firefox,Macintosh,False,desktop,Oceania,Australasia,Australia,not available in demo dataset,not available in demo dataset,dodo.net.au,377306020877927890
2,-0.136249,-1.345793,-0.373029,-0.4056,-0.113349,1,1,Organic Search,Chrome,Windows,False,desktop,Europe,Southern Europe,Spain,Community of Madrid,Madrid,unknown.unknown,3895546263509774583
3,-0.136249,-1.344039,-0.373029,-0.4056,-0.113349,1,1,Organic Search,UC Browser,Linux,False,desktop,Asia,Southeast Asia,Indonesia,not available in demo dataset,not available in demo dataset,unknown.unknown,4763447161404445595
4,-0.028533,-1.350535,-0.373029,-0.4056,-0.113349,0,1,Organic Search,Chrome,Android,True,mobile,Europe,Northern Europe,United Kingdom,not available in demo dataset,not available in demo dataset,unknown.unknown,27294437909732085


In [7]:
train_df.to_csv('train_df.csv', index=False)

In [15]:
len(df[df['transactionRevenue']!=0])

903653

In [18]:
train_df.transactionRevenue.value_counts()

-0.113349     892138
 8.195309        256
 8.250850        189
 8.535480        187
 8.679092        170
 8.083870        135
 8.790479        122
 8.276462        116
 8.165035         98
 8.139419         93
 8.256079         92
 7.977893         84
 8.824918         81
 8.387877         77
 8.968513         65
 8.429986         64
 8.429802         62
 8.541241         54
 8.622519         51
 8.881412         46
 7.125049         44
 8.458349         40
 8.324052         40
 8.569784         39
 8.485515         39
 8.212630         38
 8.061335         37
 8.444280         37
 8.485351         37
 8.511123         37
               ...  
 10.073901         1
 9.105999          1
 10.002850         1
 8.748082          1
 9.216302          1
 9.546612          1
 9.011470          1
 9.574815          1
 9.095419          1
 9.125052          1
 10.095632         1
 9.547513          1
 8.803674          1
 8.637275          1
 8.942385          1
 10.187575         1
 9.768927    